In [138]:
# Import Crime data set
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import print_function

# read in the csv file
df = pd.read_csv('data/Police_Incidents.csv')

# Reference: http://stackoverflow.com/questions/11346283/renaming-columns-in-pandas
# Removing spaces from headers

df.columns = [x.strip().replace(' ', '') for x in df.columns]
df.columns = [x.strip().replace('/', '') for x in df.columns]
df.columns = [x.strip().replace('(', '') for x in df.columns]
df.columns = [x.strip().replace(')', '') for x in df.columns]
df.columns = [x.strip().replace('-', '') for x in df.columns]

#df.head()

In [139]:
#Subset the dataset to year 2016
# Reference: http://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value

df = df[df.Year1ofOccurrence == 2016]

In [140]:
# Narrowing down our focus to the attributes mentioned in the Dataset 
# description by removing columns not used in analysis

for col in ['OffenseServiceNumber','ServiceNumberID','ReportingArea','Beat','Sector','CouncilDistrict',
            'DPDSwornMarshallsinvolved','ComplainantHomeAddress','ComplainantApartment','ComplainantZipCode',
            'ComplainantCity','ComplainantState','ComplainantBusinessName','ComplainantBusinessAddress',
            'YearAssignment','Weather','RespondingOfficer#1BadgeNo','RespondingOfficer#1Name','RespondingOfficer#2BadgeNo',
            'RespondingOfficer#2Name','ReportingOfficerBadgeNo','AssistingOfficerBadgeNo','ReviewingOfficerBadgeNo',
            'ElementNumberAssigned','InvestigatingUnit1','InvestigatingUnit2','SpecialReportPreRMS','UCRDisposition',
            'UCR1','UCR2PreRMS','FinalUCR','RMSCode','CJISCode','PenalCode','HateCrime','VictimPackage', 'UpdateDate',
            'ModusOperandiMO', 'OffenseCodeCC', 'VictimInjuryDescription', 'OffenseStatus', 
            'ComplainantAgeatOffense', 'ComplainantAge', 'ComplainantGender', 'ComplainantRace', 'ComplainantName',
           'OffenseEnteredDateTime', 'OffenseEnteredTime', 'OffenseEnteredDayoftheWeek', 'OffenseEnteredMonth', 
            'OffenseEnteredYear', 'Dateincidentcreated', 'MapDate', 'Day2oftheYear','IncidentNumberwoYear', 'YearofIncident',
           'IncidentNumberwYear', 'Call911Problem', 'StreetBlock', 'StreetDirection', 'StreetName', 
           'IncidentAddress', 'ApartmentNumber', 'ZipCode', 'City', 'State', 'XCoordinate', 'YCoordinate', 'TargetAreaActionGrids', 
           'Community', 'Date1ofOccurrence', 'Date2ofOccurrence', 'Year2ofOccurrence', 'Month2ofOccurence', 'Day2oftheWeek', 
            'Time2ofOccurrence', 'StartingDateTime', 'EndingDateTime', 'DateofReport','CallDateTime',
            'TypeofProperty', 'IncidentNumberwYear', 'Year1ofOccurrence', 'Day1oftheYear', 'TypeofIncident','UCROffenseName',
           'OffenseType','FamilyOffense','PersonInvolvementType','TypeofLocation','VictimCondition']:
    if col in df:
        del df[col]
        
#Let's get the specs of our subsetted data:
print (df.info())       

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99341 entries, 0 to 196036
Data columns (total 12 columns):
Watch                    99341 non-null object
PenaltyClass             84593 non-null object
Division                 99321 non-null object
Month1ofOccurence        99341 non-null object
Day1oftheWeek            99341 non-null object
Time1ofOccurrence        99341 non-null object
CallReceivedDateTime     99328 non-null object
CallClearedDateTime      99310 non-null object
CallDispatchDateTime     99319 non-null object
UCROffenseDescription    99341 non-null object
GangRelatedOffense       99339 non-null object
DrugRelatedIncident      99339 non-null object
dtypes: object(12)
memory usage: 9.9+ MB
None


In [141]:
df.head()

,Watch,PenaltyClass,Division,Month1ofOccurence,Day1oftheWeek,Time1ofOccurrence,CallReceivedDateTime,CallClearedDateTime,CallDispatchDateTime,UCROffenseDescription,GangRelatedOffense,DrugRelatedIncident
0,3,NaN,Central,December,Sat,23:30,12/31/2016 11:48:58 PM,01/01/2017 03:16:32 AM,12/31/2016 11:48:58 PM,CRIMINAL MISCHIEF/VANDALISM,UNK,UNK
1,3,NaN,Central,December,Sat,23:30,12/31/2016 11:48:58 PM,01/01/2017 03:16:32 AM,12/31/2016 11:48:58 PM,CRIMINAL MISCHIEF/VANDALISM,UNK,UNK
2,3,NaN,Central,December,Sat,23:30,12/31/2016 11:48:58 PM,01/01/2017 03:16:32 AM,12/31/2016 11:48:58 PM,CRIMINAL MISCHIEF/VANDALISM,UNK,UNK
3,U,MB,NorthEast,December,Thu,12:00,01/02/2017 02:39:45 PM,01/02/2017 09:59:27 PM,01/02/2017 02:39:45 PM,CRIMINAL MISCHIEF/VANDALISM,UNK,UNK
5,3,MA,SouthEast,December,Sat,23:20,12/31/2016 11:20:39 PM,01/01/2017 01:27:25 AM,12/31/2016 11:24:31 PM,ASSAULT,No,No


In [142]:
# Reference: http://stackoverflow.com/questions/16852911/how-do-i-convert-dates-in-a-pandas-data-frame-to-a-date-data-type
df['CallReceivedDateTime'] = pd.to_datetime(df['CallReceivedDateTime'])
df['CallClearedDateTime'] = pd.to_datetime(df['CallClearedDateTime'])
df['CallDispatchDateTime'] = pd.to_datetime(df['CallDispatchDateTime'])

In [143]:
# Compute the time between call received and call displatched, call Displatch and call cleared.
# Reference: http://www.itgo.me/a/x791470639852744898/calculate-pandas-dataframe-time-difference-between-two-columns-in-hours-and-minu
df['rec_disp_time_m'] = (df['CallDispatchDateTime'] - df['CallReceivedDateTime']).astype('timedelta64[m]')
df['disp_clear_time_m'] = (df['CallClearedDateTime'] - df['CallDispatchDateTime']).astype('timedelta64[m]')

# Create Hour of Occurrence from Time Occurrence 
# Ref: http://stackoverflow.com/questions/25789445/pandas-make-new-column-from-string-slice-of-another-column
df['Hour'] = df.Time1ofOccurrence.str[:2]
df['Hour'] = (df['Hour']).astype('int64')

# Retrieve panalty class as only f for felony or M misdemeanor
df['Penalty'] = df.PenaltyClass.str[:1]

# Change day of week to indicate weekend day.  
df['IsWeekend'] = (df['Day1oftheWeek'] == 'Sat') | (df['Day1oftheWeek'] == 'Fri') | (df['Day1oftheWeek'] == 'Sun')
df.IsWeekend = df.IsWeekend.astype(np.int)

# change Gang Related attribute to a more usable value
df['IsGangRelated'] = df.GangRelatedOffense == 'Yes'
df.IsGangRelated = df.IsGangRelated.astype(np.int)

# change Drug Related attribute to a more usable value
df['IsDrugRelated'] = df.DrugRelatedIncident == 'Yes'
df.IsDrugRelated = df.IsDrugRelated.astype(np.int)
        
# Remove call datetime call columns
for col in ['CallReceivedDateTime','CallClearedDateTime','CallDispatchDateTime','Time1ofOccurrence',
            'PenaltyClass','Day1oftheWeek','GangRelatedOffense','DrugRelatedIncident']:
    if col in df:
        del df[col]
        
#df.head()

In [149]:
df.to_csv('Police_Incidents_mini.csv', sep=',', encoding='utf-8')

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99341 entries, 0 to 196036
Data columns (total 11 columns):
Watch                    99341 non-null object
Division                 99321 non-null object
Month1ofOccurence        99341 non-null object
UCROffenseDescription    99341 non-null object
rec_disp_time_m          99319 non-null float64
disp_clear_time_m        99301 non-null float64
Hour                     99341 non-null int64
Penalty                  84593 non-null object
IsWeekend                99341 non-null int64
IsGangRelated            99341 non-null int64
IsDrugRelated            99341 non-null int64
dtypes: float64(2), int64(4), object(5)
memory usage: 9.1+ MB


In [147]:
# creating missing values grouping by day of week and watch
df_grouped = df.groupby(by=['Month1ofOccurence','Watch'])

# now use this grouping to fill the data set in each group, then transform back
# fill in the numeric values
df_imputed = df_grouped.transform(lambda grp: grp.fillna(grp.median()))

# fill in the categorical values
df_imputed[['Division','Penalty']] = df_grouped[['Division','Penalty']].apply(lambda grp: grp.fillna(grp.mode()))

# filling the grouped variables from original data frame
df_imputed[['Month1ofOccurence','Watch']] = df[['Month1ofOccurence','Watch']]


In [152]:
df_imputed.to_csv('Police_Incidents_imputed.csv', sep=',', encoding='utf-8')

In [148]:
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99341 entries, 0 to 196036
Data columns (total 10 columns):
rec_disp_time_m      99341 non-null float64
disp_clear_time_m    99341 non-null float64
Hour                 99341 non-null int64
IsWeekend            99341 non-null int64
IsGangRelated        99341 non-null int64
IsDrugRelated        99341 non-null int64
Division             99321 non-null object
Penalty              84594 non-null object
Month1ofOccurence    99341 non-null object
Watch                99341 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 8.3+ MB
